# Daily Pipeline

In [7]:
import hopsworks
import sys
from pathlib import Path
import warnings
from dotenv import load_dotenv
import os
from util import *
import datetime
from locations import resorts
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore", module="IPython")

In [8]:
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",             # DNS of your Hopsworks instance
    project="ID2223_Project"
)

fs = project.get_feature_store()

today = datetime.date.today().strftime('%Y-%m-%d')
warning_fg = fs.get_feature_group(
    name='avalanche_warning',
    version=4
)
weather_fg = fs.get_feature_group(
    name="weather_sensor",
    version=3
)

2025-12-30 13:18:51,183 INFO: Closing external client and cleaning up certificates.
2025-12-30 13:18:51,188 INFO: Connection closed.
2025-12-30 13:18:51,191 INFO: Initializing external client
2025-12-30 13:18:51,192 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-30 13:18:51,842 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2173


## Warnings

In [9]:
warning_data_df = warning_fg.read()
warning_data_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.43s) 


,location,latitude,longitude,date,warning_level
0,Myrkdalen Fjellandsby,60.916092,6.531734,2021-01-13 00:00:00+00:00,3
1,Hovden Alpinsenter,59.595451,7.326028,2022-09-03 00:00:00+00:00,0
2,Bjorli Ski,62.373322,8.277476,2024-06-07 00:00:00+00:00,0
3,Sauda Ski Centre,59.653477,6.223121,2025-01-09 00:00:00+00:00,2
4,Voss Resort Fjellheisar,60.666798,6.416377,2024-03-22 00:00:00+00:00,3
...,...,...,...,...,...
21858,Strandafjellet Skisenter,62.399662,6.899585,2024-12-05 00:00:00+00:00,2
21859,Rauland Skisenter,59.819122,8.161512,2025-12-28 00:00:00+00:00,1
21860,Eikedalen Ski Center AS,60.485359,5.921808,2023-08-09 00:00:00+00:00,0
21861,Galdhøpiggen Summer Ski Centre,61.748779,8.373905,2023-03-30 00:00:00+00:00,2


In [10]:
rows=[]
for location, (lat, lon) in resorts.items():
    # print(f"Fetching {location}")
    warnings = get_warning_data(today, today, lat, lon)

    for w in warnings:
        rows.append({
            "location": location,
            "latitude": lat,
            "longitude": lon,
            "date": w.get("ValidFrom"),
            "warning_level": w.get("DangerLevel")
        })
    time.sleep(0.2)


In [11]:
warning_data_today_df = pd.DataFrame(rows)
warning_data_today_df['latitude'] = warning_data_today_df['latitude'].astype('float32')
warning_data_today_df['longitude'] = warning_data_today_df['longitude'].astype('float32')
warning_data_today_df['warning_level'] = warning_data_today_df['warning_level'].astype('int32')
warning_data_today_df['date'] = pd.to_datetime(warning_data_today_df['date'], format='%Y-%m-%dT%H:%M:%S').dt.normalize()
warning_data_today_df

warning_fg.insert(warning_data_today_df)

Uploading Dataframe: 100.00% |██████████| Rows 12/12 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: avalanche_warning_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/jobs/named/avalanche_warning_4_offline_fg_materialization/executions


(Job('avalanche_warning_4_offline_fg_materialization', 'SPARK'), None)

## Weather

In [13]:
dfs = []
for loc, (lat, lon) in resorts.items():
    hourly_df = get_hourly_weather_forecast(loc, lon, lat)
    hourly_df = hourly_df.set_index('date')
    daily_df = hourly_df.between_time('11:59', '12:01')
    daily_df = daily_df.reset_index()
    dfs.append(daily_df)


weather_daily_df = pd.concat(dfs, ignore_index=True)
# conver column date to 00:00 time
weather_daily_df['date'] = weather_daily_df['date'].dt.normalize()
weather_fg.insert(weather_daily_df, wait=True)


2025-12-30 13:20:30,582 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fg/3252


Uploading Dataframe: 100.00% |██████████| Rows 84/84 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_sensor_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/jobs/named/weather_sensor_3_offline_fg_materialization/executions
2025-12-30 13:20:41,808 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-30 13:20:44,980 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-30 13:22:50,199 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-12-30 13:22:50,275 INFO: Waiting for log aggregation to finish.
2025-12-30 13:22:59,055 INFO: Execution finished successfully.


(Job('weather_sensor_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "snowfall_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 3115
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 84,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-12-30T01:20:30.000581Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column